# CONTENT BASED RECOMMENDATION SYSTEM

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
import pickle

In [2]:
data=pd.read_csv("top10K-TMDB-movies.csv")
data.head()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   title              10000 non-null  object 
 2   genre              9997 non-null   object 
 3   original_language  10000 non-null  object 
 4   overview           9987 non-null   object 
 5   popularity         10000 non-null  float64
 6   release_date       10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.2+ KB


In [4]:
data.isnull().sum()

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

In [5]:
data.describe()

,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [6]:
data.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

# Feature selection

In [7]:
movies=data[['id', 'title', 'genre','overview', ]]

In [8]:
pd.set_option("display.max_column", None)
movies.head()

,id,title,genre,overview
0,278,The Shawshank Redemption,"Drama,Crime",Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance","Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Drama,Crime","Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,"Drama,History,War",The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,"Drama,Crime",In the continuing saga of the Corleone crime f...


In [9]:
movies["tags"]= movies['genre']+" ,"+movies['overview']

C:\Users\Civil Defence\AppData\Local\Temp\ipykernel_38820\1559545644.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["tags"]= movies['genre']+" ,"+movies['overview']


In [10]:
movies.head()

,id,title,genre,overview,tags
0,278,The Shawshank Redemption,"Drama,Crime",Framed in the 1940s for the double murder of h...,"Drama,Crime ,Framed in the 1940s for the doubl..."
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance","Raj is a rich, carefree, happy-go-lucky second...","Comedy,Drama,Romance ,Raj is a rich, carefree,..."
2,238,The Godfather,"Drama,Crime","Spanning the years 1945 to 1955, a chronicle o...","Drama,Crime ,Spanning the years 1945 to 1955, ..."
3,424,Schindler's List,"Drama,History,War",The true story of how businessman Oskar Schind...,"Drama,History,War ,The true story of how busin..."
4,240,The Godfather: Part II,"Drama,Crime",In the continuing saga of the Corleone crime f...,"Drama,Crime ,In the continuing saga of the Cor..."


In [11]:
movies=movies.drop(columns=['genre','overview'])

In [12]:
movies

,id,title,tags
0,278,The Shawshank Redemption,"Drama,Crime ,Framed in the 1940s for the doubl..."
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance ,Raj is a rich, carefree,..."
2,238,The Godfather,"Drama,Crime ,Spanning the years 1945 to 1955, ..."
3,424,Schindler's List,"Drama,History,War ,The true story of how busin..."
4,240,The Godfather: Part II,"Drama,Crime ,In the continuing saga of the Cor..."
...,...,...,...
9995,10196,The Last Airbender,"Action,Adventure,Fantasy ,The story follows th..."
9996,331446,Sharknado 3: Oh Hell No!,"Action,TV Movie,Science Fiction,Comedy,Adventu..."
9997,13995,Captain America,"Action,Science Fiction,War ,During World War I..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,"Adventure,Fantasy,Action,Drama ,A man named Fa..."


In [13]:
cv=CountVectorizer(max_features=10000, stop_words='english')

In [14]:
vector=cv.fit_transform(movies["tags"].values.astype("U")).toarray()

In [15]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
vector.shape

(10000, 10000)

In [17]:
similar=cosine_similarity(vector)

In [18]:
movies[movies["title"]=="Captain America"].index[0]

9997

In [19]:
distance= sorted(list(enumerate(similar[2])), reverse=True, key=lambda vector:vector[1])
for i in distance[0:6]:
    print(movies.iloc[i[0]].title)

The Godfather
The Godfather: Part II
Blood Ties
Joker
Bomb City
Gotti


In [20]:
def recommend(movie_name):
    index=movies[movies["title"]==movie_name].index[0]
    distance= sorted(list(enumerate(similar[index])), reverse=True, key=lambda vector:vector[1])
    
    for i in distance[0:6]:
        print(movies.iloc[i[0]].title)

In [21]:
recommend("The Godfather")

The Godfather
The Godfather: Part II
Blood Ties
Joker
Bomb City
Gotti


In [23]:
pickle.dump(movies, open('movies_list.pkl', 'wb'))
pickle.dump(similar, open('similarity.pkl', 'wb'))

In [24]:
pickle.load(open('movies_list.pkl', 'rb'))

,id,title,tags
0,278,The Shawshank Redemption,"Drama,Crime ,Framed in the 1940s for the doubl..."
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance ,Raj is a rich, carefree,..."
2,238,The Godfather,"Drama,Crime ,Spanning the years 1945 to 1955, ..."
3,424,Schindler's List,"Drama,History,War ,The true story of how busin..."
4,240,The Godfather: Part II,"Drama,Crime ,In the continuing saga of the Cor..."
...,...,...,...
9995,10196,The Last Airbender,"Action,Adventure,Fantasy ,The story follows th..."
9996,331446,Sharknado 3: Oh Hell No!,"Action,TV Movie,Science Fiction,Comedy,Adventu..."
9997,13995,Captain America,"Action,Science Fiction,War ,During World War I..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,"Adventure,Fantasy,Action,Drama ,A man named Fa..."
